<a href="https://colab.research.google.com/github/shaifuss/data_science_seminar/blob/master/multiple_instance_proposal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import defaultdict
import numpy as np
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, ConvLSTM2D, Reshape, BatchNormalization, Conv3D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input, utils
from sklearn.model_selection import train_test_split

import os
import json
import pprint

BAG_SIZE = 2

In [47]:
# load stuff
review_pix_dir = r'/content/drive/My Drive/Data Science Class/review_photos2/review_photos'
review_ids_with_photos = list(os.listdir(review_pix_dir))
with open(r'/content/drive/My Drive/Data Science Class/pizza_reviews.json', 'r') as f:
    pizza_reviews = json.load(f)

pizza_reviews_by_id = {pizza_review["review_id"]: pizza_review for pizza_review in pizza_reviews}
print("There are {} candidate reviews".format(len(review_ids_with_photos)))
# filter out review ids that don't have enough pictures
for review_id in os.listdir(review_pix_dir):
  pixdir = os.path.join(review_pix_dir, review_id)
  subdir = os.path.join(review_pix_dir, pixdir)
  if len(os.listdir(subdir)) < BAG_SIZE:
    review_ids_with_photos.remove(review_id)
print("Of those, {} have enough pictures".format(len(review_ids_with_photos)))

There are 647 candidate reviews
Of those, 173 have enough pictures


In [48]:


stars_to_photos = defaultdict(lambda: [])
bag_counter = 0
for review_id_with_photos in review_ids_with_photos:
    review_path = os.path.join(review_pix_dir, review_id_with_photos)
    image_bag = []  # will produce multiple bags per review
    for i, filename in enumerate(os.listdir(review_path)):
        img_path = os.path.join(review_path, filename)
        img = image.load_img(img_path, target_size=(299, 299))
        img_array = image.img_to_array(img)
        x = np.expand_dims(img_array, axis=0)
        x = preprocess_input(x)
        
        image_bag.append(x)
        #
        if i == BAG_SIZE - 1:
          bag_counter += 1
          t = np.stack(image_bag, axis=0)
          stars_to_photos[pizza_reviews_by_id[review_id_with_photos]["stars"]].append(t)
          image_bag = []
print("Created {} bags of size {}".format(bag_counter, BAG_SIZE))


Created 173 bags of size 2


In [49]:
from tensorflow.keras.models import Sequential
#from keras.layers.convolutional import Conv3D
#from keras.layers.convolutional_recurrent import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.

seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(BAG_SIZE, 299, 299, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))

seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))

seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))

seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.add(Reshape((-1, 1)))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')
seq.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_24 (ConvLSTM2D) (None, 2, 299, 299, 40)   62080     
_________________________________________________________________
batch_normalization_24 (Batc (None, 2, 299, 299, 40)   160       
_________________________________________________________________
conv_lst_m2d_25 (ConvLSTM2D) (None, 2, 299, 299, 40)   115360    
_________________________________________________________________
batch_normalization_25 (Batc (None, 2, 299, 299, 40)   160       
_________________________________________________________________
conv_lst_m2d_26 (ConvLSTM2D) (None, 2, 299, 299, 40)   115360    
_________________________________________________________________
batch_normalization_26 (Batc (None, 2, 299, 299, 40)   160       
_________________________________________________________________
conv_lst_m2d_27 (ConvLSTM2D) (None, 2, 299, 299, 40)  

In [50]:
one_star = np.rollaxis(np.concatenate(stars_to_photos[1], axis=1),1)
two_star = np.rollaxis(np.concatenate(stars_to_photos[2], axis=1), 1)
five_star = np.rollaxis(np.concatenate(stars_to_photos[5], axis=1), 1)

X = np.concatenate([one_star] + [two_star])
X = np.concatenate([X] + [five_star])
y = np.repeat(0.0, len(stars_to_photos[1]) + len(stars_to_photos[2]))
y = np.concatenate([y, np.repeat(1.0, len(stars_to_photos[5]))]).reshape(-1, 1)
print("X dimensions: {}, y dimensions {}".format(X.shape, y.shape))
print("{} examples were created".format(X.shape[0]))

X dimensions: (107, 2, 299, 299, 3), y dimensions (107, 1)
107 examples were created


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((74, 2, 299, 299, 3), (74, 1), (33, 2, 299, 299, 3), (33, 1))

In [52]:
seq.fit(X_train, y_train, batch_size=5, validation_split=0.3, epochs=50, callbacks=[EarlyStopping(patience=15, restore_best_weights=True)])

Epoch 1/50
11/11 [==============================] - 41s 4s/step - loss: 0.8101 - val_loss: 0.6931
Epoch 2/50
11/11 [==============================] - 40s 4s/step - loss: 0.8156 - val_loss: 0.6930
Epoch 3/50
11/11 [==============================] - 40s 4s/step - loss: 0.8120 - val_loss: 0.6929
Epoch 4/50
11/11 [==============================] - 40s 4s/step - loss: 0.7948 - val_loss: 0.6928
Epoch 5/50
11/11 [==============================] - 40s 4s/step - loss: 0.7909 - val_loss: 0.6927
Epoch 6/50
11/11 [==============================] - 40s 4s/step - loss: 0.7819 - val_loss: 0.6926
Epoch 7/50
11/11 [==============================] - 40s 4s/step - loss: 0.7723 - val_loss: 0.6925
Epoch 8/50
11/11 [==============================] - 40s 4s/step - loss: 0.7796 - val_loss: 0.6924
Epoch 9/50
11/11 [==============================] - 40s 4s/step - loss: 0.7729 - val_loss: 0.6924
Epoch 10/50
11/11 [==============================] - 40s 4s/step - loss: 0.7586 - val_loss: 0.6922
Epoch 11/50
11/11 [

In [53]:
seq.evaluate(X_test, y_test)

2/2 [==============================] - 0s 116ms/step - loss: 0.6880


0.6880099773406982